<a href="https://colab.research.google.com/github/Martipetti/MachineLearning/blob/develop-martipetti/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network

In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

In [2]:
link = 'http://tennis-data.co.uk/2023/2023.xlsx'
totalData = pd.read_excel(link)

Cleaning DataSet

In [3]:
totalData = totalData[totalData['Series'] == 'Grand Slam']

In [4]:
trainingSet = totalData[["Tournament", "Surface", "Round", "Winner", "Loser", "WRank", "LRank", "Wsets", "Lsets", "B365W", "B365L", "PSW", "PSL"]]

In [5]:
trainingSet

,Tournament,Surface,Round,Winner,Loser,WRank,LRank,Wsets,Lsets,B365W,B365L,PSW,PSL
112,Australian Open,Hard,1st Round,Moutet C.,Wu Y.,62.0,114.0,3.0,2.0,2.38,1.57,2.40,1.65
113,Australian Open,Hard,1st Round,Watanuki Y.,Rinderknech A.,138.0,59.0,3.0,0.0,2.30,1.62,2.18,1.77
114,Australian Open,Hard,1st Round,Van De Zandschulp B.,Ivashka I.,34.0,73.0,3.0,1.0,1.36,3.20,1.47,2.91
115,Australian Open,Hard,1st Round,Etcheverry T.,Barrere G.,79.0,83.0,3.0,1.0,2.30,1.62,2.43,1.63
116,Australian Open,Hard,1st Round,Sinner J.,Edmund K.,16.0,581.0,3.0,0.0,1.06,10.00,1.05,15.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,US Open,Hard,Quarterfinals,Medvedev D.,Rublev A.,3.0,8.0,3.0,0.0,1.36,3.20,1.39,3.30
2221,US Open,Hard,Quarterfinals,Alcaraz C.,Zverev A.,1.0,12.0,3.0,0.0,1.20,4.50,1.22,4.90
2222,US Open,Hard,Semifinals,Djokovic N.,Shelton B.,2.0,47.0,3.0,0.0,1.06,10.00,1.09,9.76
2223,US Open,Hard,Semifinals,Medvedev D.,Alcaraz C.,3.0,1.0,3.0,1.0,4.33,1.22,4.13,1.28


In [6]:
trainingSet.reset_index(drop=True, inplace=True)
trainingSet.index = range(1, len(trainingSet) + 1)

In [7]:
nuovi_nomi_colonne = ["Tournament","Surface", "Round", "Player1", "Player2", "Rank1", "Rank2", "Sets1", "Sets2", "B1", "B2", "PS1", "PS2"]
trainingSet.columns = nuovi_nomi_colonne

In [8]:
trainingSet["target"] = 0

<ipython-input-8-60ba1f42b5c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet["target"] = 0


In [9]:
trainingSet["Tournament"] = trainingSet["Tournament"].astype("category")
trainingSet["Surface"] = trainingSet["Surface"].astype("category")
trainingSet["Round"] = trainingSet["Round"].astype("category")
trainingSet["Player1"] = trainingSet["Player1"].astype("category")
trainingSet["Player2"] = trainingSet["Player2"].astype("category")

<ipython-input-9-1d9e29e86b61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet["Tournament"] = trainingSet["Tournament"].astype("category")
<ipython-input-9-1d9e29e86b61>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet["Surface"] = trainingSet["Surface"].astype("category")
<ipython-input-9-1d9e29e86b61>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [10]:
trainingSet.dtypes

Tournament    category
Surface       category
Round         category
Player1       category
Player2       category
Rank1          float64
Rank2          float64
Sets1          float64
Sets2          float64
B1             float64
B2             float64
PS1            float64
PS2            float64
target           int64
dtype: object

In [11]:
label_encoder = LabelEncoder()
trainingSet["Tournament"] = label_encoder.fit_transform(trainingSet["Tournament"])
trainingSet["Surface"] = label_encoder.fit_transform(trainingSet["Surface"])

<ipython-input-11-23d01b9216b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet["Tournament"] = label_encoder.fit_transform(trainingSet["Tournament"])
<ipython-input-11-23d01b9216b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet["Surface"] = label_encoder.fit_transform(trainingSet["Surface"])


In [12]:
trainingSet["Player2Code"] = label_encoder.fit_transform(trainingSet["Player2"])
trainingSet["Player1Code"] = label_encoder.transform(trainingSet["Player1"])

<ipython-input-12-7327b8ca9872>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet["Player2Code"] = label_encoder.fit_transform(trainingSet["Player2"])
<ipython-input-12-7327b8ca9872>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet["Player1Code"] = label_encoder.transform(trainingSet["Player1"])


In [13]:
trainingSet

,Tournament,Surface,Round,Player1,Player2,Rank1,Rank2,Sets1,Sets2,B1,B2,PS1,PS2,target,Player2Code,Player1Code
1,0,2,1st Round,Moutet C.,Wu Y.,62.0,114.0,3.0,2.0,2.38,1.57,2.40,1.65,0,185,117
2,0,2,1st Round,Watanuki Y.,Rinderknech A.,138.0,59.0,3.0,0.0,2.30,1.62,2.18,1.77,0,144,182
3,0,2,1st Round,Van De Zandschulp B.,Ivashka I.,34.0,73.0,3.0,1.0,1.36,3.20,1.47,2.91,0,76,175
4,0,2,1st Round,Etcheverry T.,Barrere G.,79.0,83.0,3.0,1.0,2.30,1.62,2.43,1.63,0,8,49
5,0,2,1st Round,Sinner J.,Edmund K.,16.0,581.0,3.0,0.0,1.06,10.00,1.05,15.72,0,47,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2,2,Quarterfinals,Medvedev D.,Rublev A.,3.0,8.0,3.0,0.0,1.36,3.20,1.39,3.30,0,146,106
505,2,2,Quarterfinals,Alcaraz C.,Zverev A.,1.0,12.0,3.0,0.0,1.20,4.50,1.22,4.90,0,191,1
506,2,2,Semifinals,Djokovic N.,Shelton B.,2.0,47.0,3.0,0.0,1.06,10.00,1.09,9.76,0,155,43
507,2,2,Semifinals,Medvedev D.,Alcaraz C.,3.0,1.0,3.0,1.0,4.33,1.22,4.13,1.28,0,1,106


Mischio i vinvitori per non avere vincitori su una colonna e perdenti sull'altra

In [19]:
half = len(trainingSet) // 2
trainingSet_tmp1 = trainingSet.iloc[:half]
trainingSet_tmp2 = trainingSet.iloc[half:]

In [20]:
trainingSet_tmp2

,Tournament,Surface,Round,Player1,Player2,Rank1,Rank2,Sets1,Sets2,B1,B2,PS1,PS2,target,Player2Code,Player1Code
255,3,1,1st Round,Barrios M.,Baez S.,133.0,46.0,3.0,1.0,2.10,1.73,1.99,1.92,0,5,9
256,3,1,1st Round,Musetti L.,Varillas J.P.,16.0,63.0,3.0,0.0,1.06,10.00,1.07,11.93,0,177,122
257,3,1,1st Round,Karatsev A.,Van Assche L.,50.0,75.0,3.0,1.0,1.73,2.10,1.63,2.45,0,174,79
258,3,1,1st Round,Thompson J.,Nakashima B.,70.0,55.0,3.0,2.0,1.91,1.91,1.88,2.04,0,124,167
259,3,1,1st Round,Rublev A.,Purcell M.,7.0,64.0,3.0,0.0,1.06,10.00,1.08,10.52,0,140,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2,2,Quarterfinals,Medvedev D.,Rublev A.,3.0,8.0,3.0,0.0,1.36,3.20,1.39,3.30,0,146,106
505,2,2,Quarterfinals,Alcaraz C.,Zverev A.,1.0,12.0,3.0,0.0,1.20,4.50,1.22,4.90,0,191,1
506,2,2,Semifinals,Djokovic N.,Shelton B.,2.0,47.0,3.0,0.0,1.06,10.00,1.09,9.76,0,155,43
507,2,2,Semifinals,Medvedev D.,Alcaraz C.,3.0,1.0,3.0,1.0,4.33,1.22,4.13,1.28,0,1,106


In [16]:
columns_to_swap = [["Player1", "Player2"], ["Rank1", "Rank2"], ["Sets1", "Sets2"], ["B1", "B2"], ["PS1", "PS2"]]
for col in columns_to_swap:
  trainingSet_tmp2[col] = trainingSet_tmp2[col][::-1]

<ipython-input-16-8370b0cb7473>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet_tmp2[col] = trainingSet_tmp2[col][::-1]
<ipython-input-16-8370b0cb7473>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingSet_tmp2[col] = trainingSet_tmp2[col][::-1]
<ipython-input-16-8370b0cb7473>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [18]:
trainingSet_tmp2

,Tournament,Surface,Round,Player1,Player2,Rank1,Rank2,Sets1,Sets2,B1,B2,PS1,PS2,target,Player2Code,Player1Code
255,3,1,1st Round,Barrios M.,Baez S.,133.0,46.0,3.0,1.0,2.10,1.73,1.99,1.92,0,5,9
256,3,1,1st Round,Musetti L.,Varillas J.P.,16.0,63.0,3.0,0.0,1.06,10.00,1.07,11.93,0,177,122
257,3,1,1st Round,Karatsev A.,Van Assche L.,50.0,75.0,3.0,1.0,1.73,2.10,1.63,2.45,0,174,79
258,3,1,1st Round,Thompson J.,Nakashima B.,70.0,55.0,3.0,2.0,1.91,1.91,1.88,2.04,0,124,167
259,3,1,1st Round,Rublev A.,Purcell M.,7.0,64.0,3.0,0.0,1.06,10.00,1.08,10.52,0,140,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2,2,Quarterfinals,Medvedev D.,Rublev A.,3.0,8.0,3.0,0.0,1.36,3.20,1.39,3.30,0,146,106
505,2,2,Quarterfinals,Alcaraz C.,Zverev A.,1.0,12.0,3.0,0.0,1.20,4.50,1.22,4.90,0,191,1
506,2,2,Semifinals,Djokovic N.,Shelton B.,2.0,47.0,3.0,0.0,1.06,10.00,1.09,9.76,0,155,43
507,2,2,Semifinals,Medvedev D.,Alcaraz C.,3.0,1.0,3.0,1.0,4.33,1.22,4.13,1.28,0,1,106
